In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

*Loading Dataset*

In [ ]:
train = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')

In [ ]:
test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')

In [ ]:
train.head()

**Data Cleaning**

In [ ]:
train.info()

In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())

In [ ]:
train.describe()

**Visualisation**

In [ ]:
train.groupby('SmokingStatus').count()

Never smoked 429
Ex-smoker    1038 


In [ ]:
train.groupby('Sex').count()

In [ ]:
train['Sex'] = train['Sex'].map({'Male':0,'Female':1})

In [ ]:
test['Sex'] = test['Sex'].map({'Male':0,'Female':1})

In [ ]:
train.head()

In [ ]:
train = pd.get_dummies(train,columns=['SmokingStatus'],drop_first=True)

In [ ]:
test = pd.get_dummies(test,columns=['SmokingStatus'],drop_first=True)

In [ ]:
train.head()

In [ ]:
sns.pairplot(train)

In [ ]:
sns.heatmap(train.corr(),cmap = 'RdYlBu_r')

Data has a correlation with Sex vs SmokingStatus  and Percentage vs FVC

In [ ]:
cat_cols =[col for col in train.columns if train[col].dtype==object]

In [ ]:
cat_cols

In [ ]:
from sklearn.preprocessing import LabelEncoder
cat_encs = []
for col in cat_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    cat_encs.append([col,le])

In [ ]:
cat_encs

In [ ]:
cat_test = []
for col in cat_cols:
    le = LabelEncoder()
    test[col] = le.fit_transform(test[col])
    cat_test.append([col,le])

In [ ]:
cat_test

In [ ]:
X =train.drop('Patient',axis=1)
y= train['Patient']

In [ ]:
X.shape

In [ ]:
y.shape

        **Decision Tree Classifier**

In [ ]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(X,y)
y_predict=model.predict(X)
from sklearn.metrics import accuracy_score
accuracy_score(y,y_predict)


In [ ]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
clf_rf = ensemble.RandomForestClassifier(random_state=1)
parameters = { 
    'n_estimators': [100, 400],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [2, 4, 6]    
}

from sklearn.model_selection import GridSearchCV, cross_val_score

cv_rf = GridSearchCV(estimator = clf_rf, param_grid = parameters, cv=5, n_jobs=-1)

In [ ]:
X_test = test

clf = cv_rf.fit(X, y)

predictions = clf.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
output = pd.DataFrame({'Patient': test.Patient, 'FVC': predictions})
output.to_csv('submission.csv', index=False)